# 0.Описание проекта

Имеются следующие входные данные:

Таблица `users x movies` с рейтингами. Архив с датасетом скачен с сайта GroupLens. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.
Информация о структуре датасета расположена по ссылке: http://files.grouplens.org/datasets/movielens/ml-100k-README.txt

**Задача:**
В поле “hist_film” нужно указать для заданного id фильма количество поставленных оценок в следующем порядке: "1", "2", "3", "4", "5". То есть, сколько было единичек, двоек, троек и т.д.

В поле “hist_all” нужно указать то же самое, только для всех фильмов общее количество поставленных оценок в том же порядке: "1", "2", "3", "4", "5".

`id фильма` : 302

[`Автор`: Eлена Сидорова]

[`Email`: e_sidorova_94@mail.ru]

In [7]:
from IPython.display import IFrame, Image

***Импорт Spark***

In [4]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


***Настройка спарк-сессии***

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

# 1. Чтение данных

In [8]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


*Пользователи*

Посмотрим на таблицу пользователей следующего формата:

              The full u data set, 100000 ratings by 943 users on 1682 items.
              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly
              ordered. This is a tab separated list of 
	          
              -- user id | item id | rating | timestamp --

In [9]:
users = sc.textFile("/labs/laba01/ml-100k/u.data")

In [12]:
users.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [13]:
users.count()

100000

In [14]:
users.getNumPartitions()

2

Трансформируем датасет и отфильтруем необходимый id:

In [83]:
users1 = users.map(lambda x: x.split("\t"))

In [84]:
users1.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [85]:
users_filtered = users1.filter(lambda x: x[1] == '302')

In [86]:
users_filtered.take(10)

[['186', '302', '3', '891717742'],
 ['191', '302', '4', '891560253'],
 ['49', '302', '4', '888065432'],
 ['54', '302', '4', '880928519'],
 ['62', '302', '3', '879371909'],
 ['206', '302', '5', '888180227'],
 ['197', '302', '3', '891409070'],
 ['214', '302', '4', '892668197'],
 ['190', '302', '5', '891033606'],
 ['303', '302', '4', '879465986']]

*Фильмы*

Посмотрим на информацию о фильмах. Она имеет следующий формат:

              Information about the items (movies); this is a tab separated
              list of
              -- movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western | --
              
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.

In [17]:
films = sc.textFile("/labs/laba01/ml-100k/u.item")

In [18]:
films.take(10)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0',
 '6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0',
 '7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0',
 '8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title

In [19]:
films.count()

1682

In [20]:
films.getNumPartitions()

2

Трансформируем датасет и отфильтруем необходимый id:

In [40]:
films1 = films.map(lambda x: x.split("|"))

In [240]:
#films1.collect()

In [80]:
films_filtered = films1.filter(lambda x: x[0] == '302')

In [241]:
#films_filtered.take(1)

In [43]:
films_filtered.take(0)

[]

**Итог**

Сформируем количество оценок 1-2-3-4-5 для фильма 302:

In [221]:
hist_film = users.map(lambda x: x.split("\t"))\
     .filter(lambda x: x[1] == '302')\
     .map(lambda x: (x[2], 1))\
     .sortBy(lambda x: x[0])\
     .countByKey()

In [222]:
hist_film.items()

dict_items([('1', 2), ('2', 10), ('3', 46), ('4', 119), ('5', 120)])

Сформируем количество оценок 1-2-3-4-5 для всех фильмов:

In [223]:
hist_all = users.map(lambda x: x.split("\t"))\
     .map(lambda x: (x[2], 1))\
     .sortBy(lambda x: x[0])\
     .countByKey()

In [224]:
hist_all.items()

dict_items([('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)])

In [225]:
hist_film

defaultdict(int, {'1': 2, '2': 10, '3': 46, '4': 119, '5': 120})

Конвертируем результат в итоговый json файл:

In [231]:
import json
data = {
   "hist_film": [  
      2,
      10,
      46,
      119,
      120
   ],
   "hist_all": [  
      6110,
      11370,
      27145,
      34174,
      21201
   ]
}

json_data = json.dumps(data)

In [232]:
json_data

'{"hist_film": [2, 10, 46, 119, 120], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [235]:
with open("lab01.json", "w") as f:
    f.write(json.dumps(data, indent=4))

Закрываем сессию SparkContext:

In [236]:
sc.stop()